## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [1]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta, time

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen

from release_info import release_date, start_date, PYSALVER

since = datetime.combine(start_date, time(0,0))


/tmp/ipykernel_43857/300035145.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
release_date

datetime.date(2024, 1, 31)

In [3]:
CWD = os.path.abspath(os.path.curdir)

In [4]:
CWD

'/home/serge/para/1_projects/code-pysal-meta/pysal/tools'

In [5]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [6]:
packages

{'libpysal': '4.11.0',
 'access': '1.1.9',
 'esda': '2.5.1',
 'giddy': '2.3.5',
 'inequality': '1.0.1',
 'pointpats': '2.5.0',
 'segregation': '2.5',
 'spaghetti': '1.7.6',
 'mgwr': '2.2.1',
 'momepy': '0.7.2',
 'spglm': '1.1.0',
 'spint': '1.0.7',
 'spreg': '1.5.0',
 'spvcm': '0.3.0',
 'tobler': '0.11.2',
 'mapclassify': '2.7.0',
 'splot': '1.1.5.post1',
 'spopt': '0.6.1'}

import pysal
packages['pysal'] = pysal.__version__

In [7]:
import pickle

In [8]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [9]:
type(issues_closed)

dict

In [10]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [11]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [12]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [13]:
from datetime import datetime

In [14]:
#pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
pysal_date = release_date
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [15]:
pysal_rel = {'version': f'v{PYSALVER}',
            'release_date': release_date}
github_releases['pysal'] = pysal_rel

In [16]:
github_releases

{'libpysal': {'version': 'v4.11.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.11.0',
  'release_date': datetime.datetime(2024, 6, 10, 19, 39, 46)},
 'access': {'version': 'v1.1.9',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.9',
  'release_date': datetime.datetime(2023, 10, 6, 2, 43, 55)},
 'esda': {'version': 'v2.5.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.5.1',
  'release_date': datetime.datetime(2023, 10, 24, 17, 26, 14)},
 'giddy': {'version': 'v2.3.5',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.5',
  'release_date': datetime.datetime(2024, 1, 16, 22, 39, 24)},
 'inequality': {'version': 'v1.0.1',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.1',
  'release_date': datetime.datetime(2023, 10, 28, 23, 10, 33)},
 'pointpats': {'version': 'v2.5.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.5.0',
  'release_date': datetime.datetime(2024, 7, 4, 5, 5, 17)}

In [17]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2024-06-10 19:39:46 51 36 0 0
access 2023-10-06 02:43:55 0 0 0 0
esda 2023-10-24 17:26:14 9 0 0 0
giddy 2024-01-16 22:39:24 15 0 0 0
inequality 2023-10-28 23:10:33 3 0 2 0
pointpats 2024-07-04 05:05:17 9 8 6 5
segregation 2023-08-24 16:41:39 2 0 0 0
spaghetti 2024-06-21 13:02:05 12 10 0 0
mgwr 2024-01-06 16:43:47 4 0 0 0
momepy 2024-06-27 08:26:08 95 90 0 0
spglm 2023-10-25 13:50:11 2 0 2 0
spint 2020-09-09 02:28:50 0 0 0 0
spreg 2024-07-03 14:17:21 7 7 7 7
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2023-09-26 01:19:23 11 0 0 0
mapclassify 2024-07-04 05:42:00 14 14 0 0
splot 2022-04-13 21:13:27 0 0 0 0
spopt 2024-06-20 13:41:36 18 16 0 0


In [18]:
issue_details = final_issues
pull_details = final_pulls

In [19]:
packages

{'libpysal': '4.11.0',
 'access': '1.1.9',
 'esda': '2.5.1',
 'giddy': '2.3.5',
 'inequality': '1.0.1',
 'pointpats': '2.5.0',
 'segregation': '2.5',
 'spaghetti': '1.7.6',
 'mgwr': '2.2.1',
 'momepy': '0.7.2',
 'spglm': '1.1.0',
 'spint': '1.0.7',
 'spreg': '1.5.0',
 'spvcm': '0.3.0',
 'tobler': '0.11.2',
 'mapclassify': '2.7.0',
 'splot': '1.1.5.post1',
 'spopt': '0.6.1'}

In [20]:
github_releases['pysal']['release_date'] = release_date

In [21]:
released

datetime.datetime(2024, 6, 20, 13, 41, 36)

In [22]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt'])

In [23]:
spvcm = packages['spvcm']

In [24]:
## skip packages not released since last meta release

# handle meta
mrd = github_releases['pysal']['release_date']
github_releases['pysal']['release_date'] =  datetime.combine(mrd, time(0,0))


In [25]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
new:  access
new:  esda
new:  giddy
new:  inequality
new:  pointpats
new:  segregation
new:  spaghetti
new:  mgwr
new:  momepy
new:  spglm
old: spint
new:  spreg
old: spvcm
new:  tobler
new:  mapclassify
old: splot
new:  spopt
new:  pysal


In [26]:
github_releases[package]['release_date']

datetime.datetime(2024, 1, 31, 0, 0)

In [27]:
since

datetime.datetime(2023, 7, 31, 0, 0)

In [28]:
github_releases

{'libpysal': {'version': 'v4.11.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.11.0',
  'release_date': datetime.datetime(2024, 6, 10, 19, 39, 46)},
 'access': {'version': 'v1.1.9',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.9',
  'release_date': datetime.datetime(2023, 10, 6, 2, 43, 55)},
 'esda': {'version': 'v2.5.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.5.1',
  'release_date': datetime.datetime(2023, 10, 24, 17, 26, 14)},
 'giddy': {'version': 'v2.3.5',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.5',
  'release_date': datetime.datetime(2024, 1, 16, 22, 39, 24)},
 'inequality': {'version': 'v1.0.1',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.1',
  'release_date': datetime.datetime(2023, 10, 28, 23, 10, 33)},
 'pointpats': {'version': 'v2.5.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.5.0',
  'release_date': datetime.datetime(2024, 7, 4, 5, 5, 17)}

In [29]:
since_date = '--since="{start}"'.format(start=start_date.strftime("%Y-%m-%d"))
since_date
                                        

'--since="2023-07-31"'

In [30]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

2024-06-10
libpysal 554 536 2024-06-10
2023-10-06
access 0 0 2023-10-06
2023-10-24
esda 40 16 2023-10-24
2024-01-16
giddy 84 60 2024-01-16
2023-10-28
inequality 10 3 2023-10-28
2024-07-04
pointpats 114 114 2024-07-04
2023-08-24
segregation 25 11 2023-08-24
2024-06-21
spaghetti 27 26 2024-06-21
2024-01-06
mgwr 22 22 2024-01-06
2024-06-27
momepy 92 91 2024-06-27
2023-10-25
spglm 48 43 2023-10-25
2020-09-09
spint 48 0 2020-09-09
2024-07-03
spreg 42 42 2024-07-03
2020-02-02
spvcm 42 0 2020-02-02
2023-09-26
tobler 59 30 2023-09-26
2024-07-04
mapclassify 62 62 2024-07-04
2022-04-13
splot 62 0 2022-04-13
2024-06-20
spopt 73 72 2024-06-20


In [31]:
activity

{'libpysal': 536,
 'access': 0,
 'esda': 16,
 'giddy': 60,
 'inequality': 3,
 'pointpats': 114,
 'segregation': 11,
 'spaghetti': 26,
 'mgwr': 22,
 'momepy': 91,
 'spglm': 43,
 'spint': 0,
 'spreg': 42,
 'spvcm': 0,
 'tobler': 30,
 'mapclassify': 62,
 'splot': 0,
 'spopt': 72}

In [32]:
subpackage

'spopt'

In [33]:
CWD

'/home/serge/para/1_projects/code-pysal-meta/pysal/tools'

In [34]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

536
libpysal 554 536 2024-06-10
0
access 0 0 2023-10-06
16
esda 40 16 2023-10-24
60
giddy 84 60 2024-01-16
3
inequality 10 3 2023-10-28
114
pointpats 114 114 2024-07-04
11
segregation 25 11 2023-08-24
26
spaghetti 27 26 2024-06-21
22
mgwr 22 22 2024-01-06
91
momepy 92 91 2024-06-27
43
spglm 48 43 2023-10-25
42
spreg 42 42 2024-07-03
30
tobler 59 30 2023-09-26
62
mapclassify 62 62 2024-07-04
72
spopt 73 72 2024-06-20


In [35]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [36]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [37]:
author_cmd.append('blank')

In [38]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2023-07-31"', 'blank']

In [39]:
from collections import Counter

In [40]:
tag_dates

{'libpysal': '2024-06-10',
 'access': '2023-10-06',
 'esda': '2023-10-24',
 'giddy': '2024-01-16',
 'inequality': '2023-10-28',
 'pointpats': '2024-07-04',
 'segregation': '2023-08-24',
 'spaghetti': '2024-06-21',
 'mgwr': '2024-01-06',
 'momepy': '2024-06-27',
 'spglm': '2023-10-25',
 'spint': '2020-09-09',
 'spreg': '2024-07-03',
 'spvcm': '2020-02-02',
 'tobler': '2023-09-26',
 'mapclassify': '2024-07-04',
 'splot': '2022-04-13',
 'spopt': '2024-06-20'}

In [41]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--oneline', '--since="2023-07-31"']
libpysal ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2024-06-11"']
['git', 'log', '--oneline', '--since="2023-07-31"']
access ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2023-10-07"']
['git', 'log', '--oneline', '--since="2023-07-31"']
esda ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2023-10-25"']
['git', 'log', '--oneline', '--since="2023-07-31"']
giddy ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2024-01-17"']
['git', 'log', '--oneline', '--since="2023-07-31"']
inequality ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2023-10-29"']
['git', 'log', '--oneline', '--since="2023-07-31"']
pointpats ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2024-07-05"']
['git', 'log', '--oneline', '--since="2023-07-31"']
segregation ['git', 'log', '--format=* %aN', '--since="2023-07-31"', '--until="2023-08-25"']
['git', '

In [42]:
author_cmd

['git',
 'log',
 '--format=* %aN',
 '--since="2023-07-31"',
 '--until="2024-06-21"']

In [43]:
subpackage

'spopt'

In [44]:
authors_global

{b'* Anna Brazdova',
 b'* Dani Arribas-Bel',
 b'* Daniela Dan\xc4\x8dejov\xc3\xa1',
 b'* Eli Knaap',
 b'* Florian De Temmerman',
 b'* Gabriel Agostini',
 b'* Gareth Simons',
 b'* Germano Barcelos',
 b'* James Gaboardi',
 b'* Krasen Samardzhiev',
 b'* Levi John Wolf',
 b'* Lisa',
 b'* Marek Novotn\xc3\xbd',
 b'* Martin Fleischmann',
 b'* Olivier Burggraaff',
 b'* Pedro Amaral',
 b'* Robin Lenz',
 b'* Serge Rey',
 b'* Sergio Rey',
 b'* Taylor Oshan',
 b'* Wei Kang',
 b'* Ziqi Li',
 b'* anastassiavybornova',
 b'* dependabot[bot]',
 b'* eli knaap',
 b'* ggarzonie',
 b'* jGaboardi',
 b'* ljwolf',
 b'* pre-commit-ci[bot]',
 b'* rongboxu',
 b'* stevee404',
 b'* tmnj'}

In [45]:
activity

{'libpysal': 554,
 'access': 0,
 'esda': 40,
 'giddy': 84,
 'inequality': 10,
 'pointpats': 114,
 'segregation': 25,
 'spaghetti': 27,
 'mgwr': 22,
 'momepy': 92,
 'spglm': 48,
 'spint': 0,
 'spreg': 42,
 'spvcm': 0,
 'tobler': 59,
 'mapclassify': 62,
 'splot': 0,
 'spopt': 73}

In [46]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [47]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [48]:
sub

'spopt'

In [49]:
os.chdir(CWD)

import pandas

In [50]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [51]:
df.head()

,package,commits,total issues,pulls
0,libpysal,554,36,0
1,access,0,0,0
2,esda,40,0,0
3,giddy,84,0,0
4,inequality,10,0,0


In [52]:
df.shape

(18, 4)

In [53]:
type(counters)

dict

In [54]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [55]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                      1252
total issues                                                  181
pulls                                                          12
dtype: object

In [56]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [57]:
contributor_table.to_html('./contributor_table.html')

In [58]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

,commits
Anastassiavybornova,1
Anna Brazdova,1
Dani Arribas-Bel,18
Daniela Dančejová,1
Dependabot[Bot],26
Eli Knaap,154
Florian De Temmerman,3
Gabriel Agostini,1
Gareth Simons,1
Germano Barcelos,9


In [59]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [60]:
totals

Martin Fleischmann      398
Serge Rey                51
Krasen Samardzhiev       21
James Gaboardi          287
Lisa                      1
Anna Brazdova             1
Marek Novotný             1
Daniela Dančejová         1
Pre-Commit-Ci[Bot]       22
Levi John Wolf           89
Dependabot[Bot]          26
Eli Knaap               154
Florian De Temmerman      3
Robin Lenz                1
Wei Kang                 13
Jgaboardi                 1
Stevee404                 1
Ggarzonie                 1
Ziqi Li                   8
Taylor Oshan              2
Tmnj                      1
Anastassiavybornova       1
Gabriel Agostini          1
Gareth Simons             1
Pedro Amaral             19
Dani Arribas-Bel         18
Olivier Burggraaff        1
Germano Barcelos          9
Rongboxu                  1
dtype: int64

In [61]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [62]:
n_commits

1252

In [63]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues'  
    ' since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        since_date = start_date))

In [64]:
line

'Overall, there were 1252 commits that closed 181 issues since our last release on 2023-07-31.\n'

## append html files to end of changes.md with tags for toc

In [65]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    


In [66]:
df.head()

,package,commits,total issues,pulls
0,libpysal,554,36,0
1,access,0,0,0
2,esda,40,0,0
3,giddy,84,0,0
4,inequality,10,0,0


In [67]:
df.head(17)

,package,commits,total issues,pulls
0,libpysal,554,36,0
1,access,0,0,0
2,esda,40,0,0
3,giddy,84,0,0
4,inequality,10,0,0
5,pointpats,114,8,5
6,segregation,25,0,0
7,spaghetti,27,10,0
8,mgwr,22,0,0
9,momepy,92,90,0


In [68]:
n_pulls

12